In [2]:
import numpy as np
import pandas as pd

In [32]:
train_df = pd.read_csv('data/train.csv', on_bad_lines='skip')
test_df  = pd.read_csv('data/test.csv', on_bad_lines='skip')

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13177 entries, 0 to 13176
Data columns (total 1 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   	content	label	label_id  13177 non-null  object
dtypes: object(1)
memory usage: 103.1+ KB


In [5]:
train_df.head()
print(train_df.columns)
print(test_df.columns)

Index(['\tcontent\tlabel\tlabel_id'], dtype='object')
Index(['\tcontent\tlabel\tlabel_id'], dtype='object')


In [24]:
trainDf_rowCount = train_df.shape[0]
testDf_rowCount  = test_df.shape[0]

In [33]:
import re

trainTarget = np.empty(trainDf_rowCount, dtype='object')
trainData   = np.empty(trainDf_rowCount, dtype='object')

i = 0
for row in train_df['\tcontent\tlabel\tlabel_id']:
    trainTarget[i] = re.split(r'\t+', row)[-2]
    trainData[i]   = re.split(r'\t+', row)[-3]
    i += 1


In [34]:
testTarget  = np.empty(testDf_rowCount, dtype='object')
testData    = np.empty(testDf_rowCount, dtype='object')

i = 0
for row in test_df['\tcontent\tlabel\tlabel_id']:
    testTarget[i] = re.split(r'\t+', row)[-2]
    testData[i]   = re.split(r'\t+', row)[-3]
    i += 1

In [35]:
# create new dataset
trianDF = pd.DataFrame({'content': trainData, 'category': trainTarget})
testDF  = pd.DataFrame({'content': testData, 'category': testTarget})
testDF.head()

,content,category
0,حسن جوهرچی بازیگر سینما و تلویزیون ایران در گف...,فرهنگی هنری
1,به گزارش گروه بین الملل باشگاه خبرنگاران جوان ...,بین الملل
2,به گزارش خبرنگار فوتبال و فوتسال گروه ورزشی با...,ورزشی
3,به‌ گزارش گروه اقتصادی باشگاه خبرنگاران به نقل...,اقتصادی
4,به گزارش خبرنگار حوزه قرآن و عترت گروه فرهنگی ...,فرهنگی هنری


# Text Preprocessing

In [27]:
import codecs
from hazm import Normalizer

nmz = Normalizer()
stops = "\n".join(
    sorted(
        list(
            set(
                [
                    nmz.normalize(w) for w in codecs.open('persian-stopwords-master/persian', encoding='utf-8').read().split('\n') if w
                ]
            )
        )
    )
)
print(type(stops))


<class 'str'>


In [28]:
stops = np.array(stops.split('\n'))
stops

array(['"', '#', '(', ..., '…', '\ufeff!', '\ufeffو'], dtype='<U17')

In [29]:
word = "دوشنبه"
isin = word in stops 
isin

False

In [41]:
lst = np.array(['۰', '۱', '۲', '۳', '۴', '۵', '۶', '۷', '۸', '۹', '؛', ':', '،', '!', '؟', '.'])
contents = []

for i in range(trianDF.shape[0]):
    container = trianDF['content'][i]
    # remove special characters
    for char in lst:
        container = container.replace(char, "")
    # remove half-space
    container = container.replace('\u200c', " ")
    # change 2 spaces with single space
    container = container.replace('  ', " ")
    contents.append(container)


trianDF.content = contents
trianDF['content'][0]

'به گزارش خبرنگار حوزه بهداشت و درمان گروه علمی پزشکی باشگاه خبرنگاران جوان محمد آقاجانی روز دوشنبه در نشست مشترک و هم اندیشی اعضای کمیسیون برنامه بودجه و محاسبات مجلس و شورای معاونین وزارت بهداشت اظهار داشت در بیش از کلینیک ویژه در سطح کشور از ابتدای طرح تحول سلامت تا پایان سال گذشته میلیون بار ویزیت تخصصی و فوق تخصصی با دریافت بهای بسیار اندک حدود هزار تومانی برای مردم طبقات محروم و متوسط انجام شد وی به بیان مهمترین مشکلات پیش روی طرح تحول سلامت پرداخت و گفت مهمترین معضل این طرح عدم پایداری منابع است و اعتبارات وزارت بهداشت در سه سال اخیر به طور کامل محقق نشد آقاجانی تصریح کرد در سال معادل یک سوم بودجه سال به وزارت بهداشت بودجه تخصیص یافت اما خدمات حوزه سلامت همچنان با تلاش ها و زحمات جامعه پزشکی پرستاری و کادر بهداشتی و درمانی کشور بدون هیچ گونه کاستی به مردم ارائه می شود معاون درمان وزارت بهداشت تاکید کرد برای جبران کاهش بودجه وزارت بهداشت در سال سوم اجرای طرح تحول سلامت نسبت به سال اول در هزینه های به طور جدی صرفه جویی کردیم اما سال گذشته از محل هدفمندی یارانه ها نیز اعتباری دریاف